# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
import re
from scipy.stats import chi2_contingency
from itertools import combinations
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
warnings.filterwarnings('ignore')

# Read Data

In [ ]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

data_train.head()

,Latitude,Longitude,Station Code,Depth,Parameter Code,Analysis Method Code,Value Flags,Data Quality,Dataset_Source,temperature_avg,...,sampling_method,is_referenced_method,water_body_type,analytical_program,pollution_risk,testing_laboratory,validated_results,sampling_weather,regulatory_framework,land_use_impact
0,50.90280,NaN,CAN00062,NaN,OH,VMV_351,NaN,Fair,NaN,NaN,...,NaN,False,Wetland,Provincial Monitoring,Medium,National Water Research Center,False,Snowy,NaN,Agricultural Dominant
1,NaN,NaN,NaN,0.3,NaN,NaN,NaN,Fair,carbon,NaN,...,Composite Sample,True,NaN,NaN,Low,NaN,True,NaN,International Water Quality Standards,Agricultural Dominant
2,46.27117,11.42484,ITA00304,0.0,NaN,APAT3200_2003,<,Fair,mercury,-5.702201,...,Grab Sample,NaN,NaN,NaN,NaN,International Water Quality Lab,True,NaN,International Water Quality Standards,NaN
3,49.45860,-120.50400,CAN00328,0.3,Cs-Tot,VMV_3528,NaN,Fair,caesium,6.250698,...,Composite Sample,False,Reservoir,NaN,NaN,Canadian Analytical Services,True,Rainy,Fisheries Act,Agricultural Dominant
4,49.52790,-115.54920,CAN00204,0.3,Li-Tot,VMV_1919,NaN,Good,lithium,-9.855316,...,Manual Sample,False,River,NaN,Medium,National Water Research Center,True,NaN,Canadian Environmental Protection Act,NaN


In [ ]:
data_train[data_train.duplicated()].shape[0]

0

In [ ]:
len(data_train)

13998

In [ ]:
data_train['Data Quality'].value_counts()

,count
Data Quality,
Fair,10133
Unknown,3590
Good,251
Pending review,24


## Train Test Split

In [ ]:
X = data_train.drop('Data Quality', axis=1)
y = data_train['Data Quality']

In [ ]:
# Stratification will be used to handle the class imbalance
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27, stratify=y)

In [ ]:
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [ ]:
test.isnull().sum()

,0
Latitude,704
Longitude,1382
Station Code,1082
Depth,895
Parameter Code,332
Analysis Method Code,2321
Value Flags,2376
Dataset_Source,1245
temperature_avg,907
rainfall_mm,992


# EDA

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11198 entries, 5474 to 4297
Data columns (total 42 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Latitude              8292 non-null   float64
 1   Longitude             5507 non-null   float64
 2   Station Code          6920 non-null   object 
 3   Depth                 7648 non-null   float64
 4   Parameter Code        9833 non-null   object 
 5   Analysis Method Code  1922 non-null   object 
 6   Value Flags           1746 non-null   object 
 7   Dataset_Source        6231 non-null   object 
 8   temperature_avg       7588 non-null   float64
 9   rainfall_mm           7016 non-null   float64
 10  humidity_pct          10512 non-null  float64
 11  pressure_hPa          5736 non-null   float64
 12  drought_index         6390 non-null   float64
 13  extreme_precip_days   9557 non-null   float64
 14  urban_pct             9670 non-null   float64
 15  forest_pct            

In [ ]:
print(pd.concat([
    train.isnull().sum(),
    (train.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                       2906       25.95
Longitude                      5691       50.82
Station Code                   4278       38.20
Depth                          3550       31.70
Parameter Code                 1365       12.19
Analysis Method Code           9276       82.84
Value Flags                    9452       84.41
Dataset_Source                 4967       44.36
temperature_avg                3610       32.24
rainfall_mm                    4182       37.35
humidity_pct                    686        6.13
pressure_hPa                   5462       48.78
drought_index                  4808       42.94
extreme_precip_days            1641       14.65
urban_pct                      1528       13.65
forest_pct                     1479       13.21
agriculture_pct                2099       18.74
water_pct                      3227       28.82
wetland_pct                    2713       24.23
grassland_pct                  2072     

In [ ]:
train['value_with_unit'].value_counts()

,count
value_with_unit,
0.0 mg/l,336
0.001 mg/l,285
0.2 µg/l,181
0.00154 mg/l,90
0.0002 mg/l,85
...,...
6.92 µg/l,1
217.99 mg/l,1
171.0 µg/l,1


In [ ]:
# Function to convert different concentration units into a consistent format: µg/L
def to_ug_per_l(text):
    if pd.isna(text):
        return None

    # Use regex to separate numeric value and unit (e.g., "0.001 mg/l" -> ("0.001", "mg/l"))
    match = re.match(r"([<>~]?\s*[\d\.eE\-]+)\s*(.+)", str(text)) 
    if not match:
        return None

    raw_val, unit = match.groups()
    num_val = re.sub(r"[<>~\s]", "", raw_val)

    try:
        num_val = float(num_val)
    except:
        return None

    unit = unit.strip().lower()

    if unit in ['mg/l', 'mg\\l']:
        return num_val * 1000
    elif unit in ['µg/l', 'ug/l', 'μg/l']:
        return num_val
    else:
        return None

train["value_ug_per_l"] = train["value_with_unit"].apply(to_ug_per_l)
test["value_ug_per_l"] = test["value_with_unit"].apply(to_ug_per_l)
data_test["value_ug_per_l"] = data_test["value_with_unit"].apply(to_ug_per_l)

## Drop Unused Column (High Missing Percentages)

In [ ]:
col_to_drop = ['Analysis Method Code', 'Value Flags', "value_with_unit"]

train.drop(col_to_drop, axis=1, inplace=True)
test.drop(col_to_drop, axis=1, inplace=True)
data_test.drop(col_to_drop, axis=1, inplace=True)

In [ ]:
def cat_num(df):
  cat = []
  num = []
  for col in df.columns:
    if df[col].dtype == 'object':
      cat.append(col)
    else:
      num.append(col)
  return cat, num

cat, num = cat_num(train)

In [ ]:
def plot_distribution(data, test, column=None):
  '''
  This function will visualizes the distribution of a numerical column
  by comparing the distribtuion on train, test dataset by using KDE plots and boxplot

  args:
    data : pandas dataframe (train data).
    test : pandas dataframe (test data).
    column : string (numerical column that want to be analyze)

  return:
    display of KDE plot and boxplot for the given column.
  '''

  cols = [column]

  df = pd.concat([data[cols].assign(Source='Train'),
                  test[cols].assign(Source='Test')])

  fig, axes = plt.subplots(len(cols), 3, figsize=(15, len(cols) * 3.2),
                           gridspec_kw={'hspace': 0.35, 'wspace': 0.3, 'width_ratios': [0.80, 0.20, 0.2]})

  if len(cols) == 1:
    axes = [axes]

  for i, col in enumerate(cols):
    #KDE Plot
    ax = axes[i][0]
    sns.kdeplot(data=df[[col, 'Source']], x=col, hue='Source', ax=ax, linewidth=2.1)
    ax.set_title(f"\n{col}", fontsize=9, fontweight='bold')
    ax.grid(visible=True, which='both', linestyle='--', color='lightgrey', linewidth=0.75)
    ax.set(xlabel='', ylabel='')

    #Boxplots
    ax = axes[i][1]
    sns.boxplot(data=df.loc[df.Source == 'Train', [col]], y=col, width=0.25, saturation=0.90, linewidth=0.90,
                fliersize=2.25, color='#037d97', ax=ax)
    ax.set(xlabel='', ylabel='')
    ax.set_title(f"Train", fontsize=9, fontweight='bold')

    ax = axes[i][2]
    sns.boxplot(data=df.loc[df.Source == 'Test', [col]], y=col, width=0.25, fliersize=2.25, saturation=0.6,
                linewidth=0.90, color='#E4591E', ax=ax)
    ax.set(xlabel='', ylabel='')
    ax.set_title(f"Test", fontsize=9, fontweight='bold')

  plt.suptitle(f"Distribution analysis - continuous columns", fontsize=12, fontweight='bold',
               y=1, x=0.5)

  #0.89 , 0.57 | 1 , 0.5
  plt.tight_layout()
  plt.show()
  return

In [ ]:
print(pd.concat([
    train.isnull().sum(),
    (train.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                       2906       25.95
Longitude                      5691       50.82
Station Code                   4278       38.20
Depth                          3550       31.70
Parameter Code                 1365       12.19
Dataset_Source                 4967       44.36
temperature_avg                3610       32.24
rainfall_mm                    4182       37.35
humidity_pct                    686        6.13
pressure_hPa                   5462       48.78
drought_index                  4808       42.94
extreme_precip_days            1641       14.65
urban_pct                      1528       13.65
forest_pct                     1479       13.21
agriculture_pct                2099       18.74
water_pct                      3227       28.82
wetland_pct                    2713       24.23
grassland_pct                  2072       18.50
barren_pct                     3642       32.52
frag_index                     1225     

## Geographic

In [ ]:
train_geo = train.copy()
test_geo = test.copy()
data_test_geo = data_test.copy()

* latitude : Geographic latitude of the sampling location.
* longitude : Geographic longitude of the sampling location.
* Station Code : Unique identifier of the monitoring station (often country-specific).
* country_code : Country where the station is located.
* testing_laboratory : Facility that performed the analysis.
* regulatory_framework : Standard or legal framework governing limit values (e.g. International Water Quality Standards).

In [ ]:
geographic_cols = ['Latitude', 'Longitude', 'Station Code', 'country_code', 'testing_laboratory', 'regulatory_framework']

train_geo[geographic_cols].head()

,Latitude,Longitude,Station Code,country_code,testing_laboratory,regulatory_framework
5474,NaN,NaN,NaN,IND,International Water Quality Lab,International Water Quality Standards
2656,52.400650,NaN,NLD00136,NLD,International Water Quality Lab,NaN
12646,50.300833,NaN,CAN00085,CAN,Provincial Analytical Lab,Canadian Environmental Protection Act
4496,44.763600,NaN,NaN,NaN,Provincial Analytical Lab,Provincial Water Regulations
13297,16.417300,-91.26377,MEX02533,NaN,International Water Quality Lab,International Water Quality Standards


In [ ]:
train_geo[geographic_cols].dtypes

,0
Latitude,float64
Longitude,float64
Station Code,object
country_code,object
testing_laboratory,object
regulatory_framework,object


In [ ]:
# Compute Cramér’s V (a measure of association between two categorical variables)
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k - 1)*(r - 1)) / (n - 1))
    rcorr = r - ((r - 1)**2) / (n - 1)
    kcorr = k - ((k - 1)**2) / (n - 1)
    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))

In [ ]:
categorical_geo_cols = [col for col in geographic_cols if train_geo[col].dtype == 'object' or train_geo[col].nunique() < 20]

results = []

for col1, col2 in combinations(categorical_geo_cols, 2):
    df_valid = train_geo[[col1, col2]].dropna()
    if df_valid.empty:
        continue
    v = cramers_v(df_valid[col1], df_valid[col2])
    results.append((col1, col2, round(v, 3)))

pd.DataFrame(results, columns=['Column 1', 'Column 2', "Cramér's V"]).sort_values(by="Cramér's V", ascending=False)

,Column 1,Column 2,Cramér's V
0,Station Code,country_code,0.660
5,testing_laboratory,regulatory_framework,0.596
4,country_code,regulatory_framework,0.565
3,country_code,testing_laboratory,0.543
2,Station Code,regulatory_framework,0.379
1,Station Code,testing_laboratory,0.352


In [ ]:
print(pd.concat([
    train[geographic_cols].isnull().sum(),
    (train[geographic_cols].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                       2906       25.95
Longitude                      5691       50.82
Station Code                   4278       38.20
country_code                   3564       31.83
testing_laboratory             2067       18.46
regulatory_framework           2780       24.83


From above we get that we can impute 'testing_laboratory' by using 'regulatory_framework' data

In [ ]:
def fill_by_group_mode(df, col1, col2, inplace=True):
    """
    Mengisi nilai NaN di salah satu kolom berdasarkan modus dari kolom lain.
    Otomatis memilih kolom dengan missing lebih sedikit sebagai dasar groupby.
    """
    # Hitung jumlah missing
    na1 = df[col1].isna().sum()
    na2 = df[col2].isna().sum()

    # Pilih kolom yang lebih sedikit NaN sebagai dasar groupby
    if na1 < na2:
        group_col, target_col = col1, col2
    else:
        group_col, target_col = col2, col1

    # Buat mapping: group_col → modus target_col
    mapping = (
        df.dropna(subset=[group_col, target_col])
          .groupby(group_col)[target_col]
          .agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
          .to_dict()
    )

    # Buat mask baris yang target_col-nya kosong tapi group_col-nya tersedia
    mask = df[target_col].isna() & df[group_col].notna()

    # Isi nilainya
    filled_values = df.loc[mask, group_col].map(mapping)

    if inplace:
        df.loc[mask, target_col] = filled_values
    else:
        df_copy = df.copy()
        df_copy.loc[mask, target_col] = filled_values
        return df_copy

    return None  # jika inplace

In [ ]:
fill_by_group_mode(train_geo, 'regulatory_framework', 'testing_laboratory')

In [ ]:
print(pd.concat([
    train_geo[geographic_cols].isnull().sum(),
    (train_geo[geographic_cols].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                       2906       25.95
Longitude                      5691       50.82
Station Code                   4278       38.20
country_code                   3564       31.83
testing_laboratory             2067       18.46
regulatory_framework            488        4.36


In [ ]:
train_geo[geographic_cols].head()

,Latitude,Longitude,Station Code,country_code,testing_laboratory,regulatory_framework
5474,NaN,NaN,NaN,IND,International Water Quality Lab,International Water Quality Standards
2656,52.400650,NaN,NLD00136,NLD,International Water Quality Lab,International Water Quality Standards
12646,50.300833,NaN,CAN00085,CAN,Provincial Analytical Lab,Canadian Environmental Protection Act
4496,44.763600,NaN,NaN,NaN,Provincial Analytical Lab,Provincial Water Regulations
13297,16.417300,-91.26377,MEX02533,NaN,International Water Quality Lab,International Water Quality Standards


In [ ]:
# Ambil 3 huruf pertama Station Code
station_prefix = train_geo['Station Code'].str[:3]

# Bandingkan dengan country_code
match_mask = (station_prefix == train_geo['country_code'])

# Hitung proporsi yang cocok
total_valid = train_geo['Station Code'].notna() & train_geo['country_code'].notna()
match_ratio = match_mask[total_valid].mean()

print(f"Proporsi Station Code prefix yang cocok dengan country_code: {match_ratio:.2%}")

Proporsi Station Code prefix yang cocok dengan country_code: 100.00%


Since the Station Code already contains part of the country_code information, we will keep it and use it to help impute some missing data.

In [ ]:
# Mengisi country_code yang hilang atau belum tersedia, dengan asumsi bahwa kode negara sudah terkandung dalam Station Code (misalnya kode ISO seperti USA, CAN, DEU, dll).
train_geo.loc[train['Station Code'].notna(), 'country_code'] = train_geo['Station Code'].str[:3]
test_geo.loc[test['Station Code'].notna(), 'country_code'] = test_geo['Station Code'].str[:3]
data_test_geo.loc[data_test['Station Code'].notna(), 'country_code'] = data_test_geo['Station Code'].str[:3]

In [ ]:
# Since we already have country_code, we might not need Station Code, so it can be dropped.
col_to_drop = ['Station Code']

In [ ]:
train_geo.drop(col_to_drop, axis=1, inplace=True)
test_geo.drop(col_to_drop, axis=1, inplace=True)
data_test_geo.drop(col_to_drop, axis=1, inplace=True)
geographic_cols.remove('Latitude')
geographic_cols.remove('Longitude')
geographic_cols.remove('Station Code')

In [ ]:
print(pd.concat([
    train_geo[geographic_cols].isnull().sum(),
    (train_geo[geographic_cols].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
country_code                   1345       12.01
testing_laboratory             2067       18.46
regulatory_framework            488        4.36


In [ ]:
categorical_geo_cols = [col for col in geographic_cols if train_geo[col].dtype == 'object' or train_geo[col].nunique() < 20]

results = []

for col1, col2 in combinations(categorical_geo_cols, 2):
    df_valid = train_geo[[col1, col2]].dropna()
    if df_valid.empty:
        continue
    v = cramers_v(df_valid[col1], df_valid[col2])
    results.append((col1, col2, round(v, 3)))

pd.DataFrame(results, columns=['Column 1', 'Column 2', "Cramér's V"]).sort_values(by="Cramér's V", ascending=False)

,Column 1,Column 2,Cramér's V
2,testing_laboratory,regulatory_framework,0.622
1,country_code,regulatory_framework,0.569
0,country_code,testing_laboratory,0.544


In [ ]:
print(pd.concat([
    train_geo[geographic_cols].isnull().sum(),
    (train_geo[geographic_cols].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
country_code                   1345       12.01
testing_laboratory             2067       18.46
regulatory_framework            488        4.36


### country_code / testing_laboratory

In [ ]:
fill_by_group_mode(train_geo, 'country_code', 'regulatory_framework')

### regulatory_framework

In [ ]:
regula_mode = train_geo['regulatory_framework'].mode()[0]

train_geo['regulatory_framework'] = train_geo['regulatory_framework'].fillna(regula_mode)
test_geo['regulatory_framework'] = test_geo['regulatory_framework'].fillna(regula_mode)
data_test_geo['regulatory_framework'] = data_test_geo['regulatory_framework'].fillna(regula_mode)

In [ ]:
lab_mode = train_geo['testing_laboratory'].mode()[0]

train_geo['testing_laboratory'] = train_geo['testing_laboratory'].fillna(lab_mode)
test_geo['testing_laboratory'] = test_geo['testing_laboratory'].fillna(lab_mode)
data_test_geo['testing_laboratory'] = data_test_geo['testing_laboratory'].fillna(lab_mode)

In [ ]:
print(pd.concat([
    train_geo[geographic_cols].isnull().sum(),
    (train_geo[geographic_cols].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
country_code                     59        0.53
testing_laboratory                0        0.00
regulatory_framework              0        0.00


In [ ]:
fill_by_group_mode(train_geo, 'country_code', 'regulatory_framework')

### Latitude / Longitude

We can impute the missing values by grouping on country_code and replacing them with the median latitude and longitude

In [ ]:
map_lat_long = train_geo.groupby('country_code')[['Latitude', 'Longitude']].median().to_dict()
map_lat_long

{'Latitude': {'ARG': -33.883055,
  'AUT': 47.670204999999996,
  'BEL': 50.96996,
  'BGR': 41.41696,
  'CAN': 50.0267,
  'CHE': 47.34652,
  'CZE': 49.84732,
  'DNK': 55.303885,
  'ESP': 41.73921,
  'EST': 59.44974,
  'FIN': 63.1691,
  'FRA': 48.48467,
  'GBR': 52.0078,
  'GRC': 38.88457,
  'HRV': 45.36497,
  'HUN': 47.1921,
  'IND': 22.73583333,
  'IRL': 52.91071,
  'ISL': 64.7109,
  'ITA': 44.88901,
  'LTU': 55.27257,
  'LVA': 57.04211,
  'MEX': 21.204720000000002,
  'MKD': 41.62,
  'NLD': 52.23708,
  'NOR': 60.37263,
  'POL': 50.23609,
  'PRT': nan,
  'ROU': 45.55288,
  'SRB': 44.73182,
  'SVN': 45.89206,
  'SWE': 59.88663,
  'URY': -33.756652,
  'USA': 38.71},
 'Longitude': {'ARG': -64.1267775,
  'AUT': 13.6495,
  'BEL': 4.302745,
  'BGR': 23.31832,
  'CAN': -116.5794,
  'CHE': 8.26683,
  'CZE': 15.882954999999999,
  'DNK': 10.48752,
  'ESP': 2.55947,
  'EST': 25.17847,
  'FIN': 25.95674,
  'FRA': 4.57904,
  'GBR': -2.680575,
  'GRC': 22.73948,
  'HRV': 16.95645,
  'HUN': 18.79785,
 

In [ ]:
# For records that still have missing longitude and latitude, we use external data (https://gist.github.com/metal3d/5b925077e66194551df949de64e910f6)
map_lat_long['Latitude']['PRT'] = 39.5
map_lat_long['Longitude']['ISL'] = -18

In [ ]:
train_geo['Latitude'] = train_geo.apply(
    lambda row: row['Latitude'] if pd.notnull(row['Latitude'])
      else map_lat_long['Latitude'].get(row['country_code'], np.nan),
    axis=1
)

train_geo['Longitude'] = train_geo.apply(
    lambda row: row['Longitude'] if pd.notnull(row['Longitude'])
      else map_lat_long['Longitude'].get(row['country_code'], np.nan),
    axis=1
)

In [ ]:
print(pd.concat([
    train_geo[geographic_cols].isnull().sum(),
    (train_geo[geographic_cols].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
country_code                      0         0.0
testing_laboratory                0         0.0
regulatory_framework              0         0.0


In [ ]:
# Mengisi nilai Latitude dan Longitude yang hilang di test_geo dan data_test_geo berdasarkan median posisi geografis tiap country_code (map_lat_long)

test_geo['Latitude'] = test_geo.apply(
    lambda row: row['Latitude'] if pd.notnull(row['Latitude'])
      else map_lat_long['Latitude'].get(row['country_code'], np.nan),
    axis=1
)

test_geo['Longitude'] = test_geo.apply(
    lambda row: row['Longitude'] if pd.notnull(row['Longitude'])
      else map_lat_long['Longitude'].get(row['country_code'], np.nan),
    axis=1
)

data_test_geo['Latitude'] = data_test_geo.apply(
    lambda row: row['Latitude'] if pd.notnull(row['Latitude'])
      else map_lat_long['Latitude'].get(row['country_code'], np.nan),
    axis=1
)

data_test_geo['Longitude'] = data_test_geo.apply(
    lambda row: row['Longitude'] if pd.notnull(row['Longitude'])
      else map_lat_long['Longitude'].get(row['country_code'], np.nan),
    axis=1
)

In [ ]:
print(pd.concat([
    test_geo[geographic_cols].isnull().sum(),
    (test_geo[geographic_cols].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
country_code                    341       12.18
testing_laboratory                0        0.00
regulatory_framework              0        0.00


## Time

In [ ]:
train_season = train_geo.copy()
test_season = test_geo.copy()
data_test_season = data_test_geo.copy()

time_col = ['time_season']

In [ ]:
train_season['time_season'].head()

,time_season
5474,1993-11-01 00:00:00 - Fall
2656,2015-03-16 00:00:00 - Spring
12646,2019-04-02 10:30:00 - Spring
4496,2017-08-28 13:40:00 - Summer
13297,2015-04-22 12:00:00 - Spring


In [ ]:
train_season[['timestamp', 'season']] = train_geo['time_season'].str.extract(r'^(.*?) - (.*)$')
test_season[['timestamp', 'season']] = test_geo['time_season'].str.extract(r'^(.*?) - (.*)$')
data_test_season[['timestamp', 'season']] = data_test_geo['time_season'].str.extract(r'^(.*?) - (.*)$')

In [ ]:
train_season.drop(['time_season'], axis=1, inplace=True)
test_season.drop(['time_season'], axis=1, inplace=True)
data_test_season.drop(['time_season'], axis=1, inplace=True)

In [ ]:
train_season[['timestamp', 'season']].head()

,timestamp,season
5474,1993-11-01 00:00:00,Fall
2656,2015-03-16 00:00:00,Spring
12646,2019-04-02 10:30:00,Spring
4496,2017-08-28 13:40:00,Summer
13297,2015-04-22 12:00:00,Spring


In [ ]:
train_season[['year', 'month', 'day']] = train_season['timestamp'].str.split('-', expand=True)
test_season[['year', 'month', 'day']] = test_season['timestamp'].str.split('-', expand=True)
data_test_season[['year', 'month', 'day']] = data_test_season['timestamp'].str.split('-', expand=True)

In [ ]:
train_season.drop(['day', 'timestamp'], axis=1, inplace=True)
test_season.drop(['day', 'timestamp'], axis=1, inplace=True)
data_test_season.drop(['day', 'timestamp'], axis=1, inplace=True)

In [ ]:
time_col = ['year', 'month', 'season']

In [ ]:
print(pd.concat([
    train_season[time_col].isnull().sum(),
    (train_season[time_col].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

        Missing count  Percentage
year              805        7.19
month             805        7.19
season            805        7.19


In [ ]:
season_mode = train_season['season'].mode()[0]
month_mode = train_season['month'].mode()[0]
year_mode = train_season['year'].mode()[0]

In [ ]:
train_season['season'] = train_season['season'].fillna(season_mode)
train_season['month'] = train_season['month'].fillna(month_mode)
train_season['year'] = train_season['year'].fillna(year_mode)

test_season['season'] = test_season['season'].fillna(season_mode)
test_season['month'] = test_season['month'].fillna(month_mode)
test_season['year'] = test_season['year'].fillna(year_mode)

data_test_season['season'] = data_test_season['season'].fillna(season_mode)
data_test_season['month'] = data_test_season['month'].fillna(month_mode)
data_test_season['year'] = data_test_season['year'].fillna(year_mode)

## pct

In [ ]:
train_pct = train_season.copy()
test_pct = test_season.copy()
data_test_pct = data_test_season.copy()

In [ ]:
pct_cols = ['humidity_pct', 'urban_pct', 'forest_pct', 'agriculture_pct', 'water_pct', 'wetland_pct', 'grassland_pct', 'barren_pct']

In [ ]:
train_pct[pct_cols].head()

,humidity_pct,urban_pct,forest_pct,agriculture_pct,water_pct,wetland_pct,grassland_pct,barren_pct
5474,78.738071,10.660415,40.474180,24.371905,NaN,8.498993,NaN,2.374052
2656,94.566138,6.793957,29.571703,27.212383,4.508927,9.743684,16.780976,5.388371
12646,79.558403,14.815609,24.819429,NaN,6.814526,9.689450,14.939614,NaN
4496,77.847505,2.301347,32.340726,NaN,9.463693,6.883121,14.464039,NaN
13297,65.971375,13.995806,33.430913,21.893808,4.840287,9.201177,10.521768,6.116241


humidity

In [ ]:
humidity_median = train_pct.groupby('country_code')['humidity_pct'].agg(lambda x: x.median()).to_dict()
humidity_median

{'ARG': 73.55681058835569,
 'AUT': 89.97626645918652,
 'BEL': 85.62818701424688,
 'BGR': 78.22537926792481,
 'CAN': 86.1459993762673,
 'CHE': 84.05237530057553,
 'CZE': 89.23048425920918,
 'DNK': 94.45459799436608,
 'ESP': 84.60974823667462,
 'EST': 86.1014685505034,
 'FIN': 87.61393816901793,
 'FRA': 85.44864292466387,
 'GBR': 81.68289258214128,
 'GRC': 83.11307793121033,
 'HRV': 87.27583318291562,
 'HUN': 89.57577386044561,
 'IND': 76.8523659922056,
 'IRL': 79.84995960845299,
 'ISL': 70.33921613522195,
 'ITA': 86.58784534637475,
 'LTU': 89.73863187729575,
 'LVA': 90.55637273875128,
 'MEX': 75.37047354267973,
 'MKD': 84.35301332038023,
 'NLD': 86.27939540089068,
 'NOR': 89.43544181713773,
 'POL': 90.43529914130482,
 'PRT': 67.57699967357905,
 'ROU': 85.2159742389892,
 'SRB': 87.09400667484306,
 'SVN': 92.47011016997408,
 'SWE': 89.76159505052783,
 'URY': 77.9026210689163,
 'USA': 77.94632554586796}

In [ ]:
train_pct['humidity_pct'] = train_pct['humidity_pct'].fillna(train_pct['country_code'].map(humidity_median))
test_pct['humidity_pct'] = test_pct['humidity_pct'].fillna(test_pct['country_code'].map(humidity_median))
data_test_pct['humidity_pct'] = data_test_pct['humidity_pct'].fillna(data_test_pct['country_code'].map(humidity_median))

In [ ]:
print(pd.concat([
    train_pct[pct_cols].isnull().sum(),
    (train_pct[pct_cols].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                 Missing count  Percentage
humidity_pct                 0        0.00
urban_pct                 1528       13.65
forest_pct                1479       13.21
agriculture_pct           2099       18.74
water_pct                 3227       28.82
wetland_pct               2713       24.23
grassland_pct             2072       18.50
barren_pct                3642       32.52


urban

In [ ]:
urban_median = train_pct.groupby('country_code')['urban_pct'].agg(lambda x: x.median()).to_dict()
urban_median

{'ARG': 4.687088597370545,
 'AUT': 8.805976808140247,
 'BEL': 5.044454918731628,
 'BGR': 12.346922140894655,
 'CAN': 5.9808750139402,
 'CHE': 5.898371749775416,
 'CZE': 10.307166486386365,
 'DNK': 4.958730688043854,
 'ESP': 3.925063567392064,
 'EST': 11.304074659443014,
 'FIN': 11.760057829592384,
 'FRA': 5.295715642087616,
 'GBR': 4.043625826696653,
 'GRC': 13.260218761260036,
 'HRV': 10.91348097195724,
 'HUN': 11.738492315331303,
 'IND': 9.923956747450134,
 'IRL': 6.134815287129956,
 'ISL': 10.41209903713543,
 'ITA': 7.479306751918829,
 'LTU': 12.040665854701873,
 'LVA': 12.539393200371997,
 'MEX': 11.752656219919034,
 'MKD': 12.895870302006488,
 'NLD': 4.984096120358627,
 'NOR': 6.508422246849573,
 'POL': 12.020338469236084,
 'PRT': 5.6919100680639465,
 'ROU': 14.070692325806576,
 'SRB': 12.918395567751771,
 'SVN': 9.293259077034715,
 'SWE': 8.409708263260242,
 'URY': 5.24423446798038,
 'USA': 11.886454103279345}

In [ ]:
train_pct['urban_pct'] = train_pct['urban_pct'].fillna(train_pct['country_code'].map(urban_median))
test_pct['urban_pct'] = test_pct['urban_pct'].fillna(test_pct['country_code'].map(urban_median))
data_test_pct['urban_pct'] = data_test_pct['urban_pct'].fillna(data_test_pct['country_code'].map(urban_median))

In [ ]:
print(pd.concat([
    train_pct[pct_cols].isnull().sum(),
    (train_pct[pct_cols].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                 Missing count  Percentage
humidity_pct                 0        0.00
urban_pct                    0        0.00
forest_pct                1479       13.21
agriculture_pct           2099       18.74
water_pct                 3227       28.82
wetland_pct               2713       24.23
grassland_pct             2072       18.50
barren_pct                3642       32.52


forest_pct

In [ ]:
forest_median = train_pct.groupby('country_code')['forest_pct'].agg(lambda x: x.median()).to_dict()
forest_median

{'ARG': 37.3241865908463,
 'AUT': 28.85018146713789,
 'BEL': 30.37788577815598,
 'BGR': nan,
 'CAN': 30.095851155025464,
 'CHE': 29.79628985228831,
 'CZE': 29.063140386394974,
 'DNK': 31.234341049635663,
 'ESP': 33.276990818881266,
 'EST': 31.36945179666808,
 'FIN': 30.64718789692573,
 'FRA': 30.58572565604459,
 'GBR': 31.644378721043765,
 'GRC': 29.872364605788285,
 'HRV': 29.18965183585746,
 'HUN': 27.639265358097624,
 'IND': 38.704719582388904,
 'IRL': 30.984160559502328,
 'ISL': nan,
 'ITA': 30.55915582266828,
 'LTU': 26.99381309020423,
 'LVA': 29.029836153973733,
 'MEX': 35.72139703876863,
 'MKD': 29.314584651473982,
 'NLD': 30.317138973727904,
 'NOR': 32.70501122896265,
 'POL': 28.07424117734579,
 'PRT': 31.273446576666245,
 'ROU': 29.546925191094587,
 'SRB': 28.07494888809012,
 'SVN': 29.020459046702268,
 'SWE': 31.522290995323324,
 'URY': 38.05363678676888,
 'USA': 31.62357166480949}

In [ ]:
train_pct['forest_pct'] = train_pct['forest_pct'].fillna(train_pct['country_code'].map(forest_median))
test_pct['forest_pct'] = test_pct['forest_pct'].fillna(test_pct['country_code'].map(forest_median))
data_test_pct['forest_pct'] = data_test_pct['forest_pct'].fillna(data_test_pct['country_code'].map(forest_median))

In [ ]:
forest_median = train_pct['forest_pct'].median()
train_pct['forest_pct'] = train_pct['forest_pct'].fillna(forest_median)
test_pct['forest_pct'] = test_pct['forest_pct'].fillna(forest_median)
data_test_pct['forest_pct'] = data_test_pct['forest_pct'].fillna(forest_median)

In [ ]:
print(pd.concat([
    train_pct[pct_cols].isnull().sum(),
    (train_pct[pct_cols].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                 Missing count  Percentage
humidity_pct                 0        0.00
urban_pct                    0        0.00
forest_pct                   0        0.00
agriculture_pct           2099       18.74
water_pct                 3227       28.82
wetland_pct               2713       24.23
grassland_pct             2072       18.50
barren_pct                3642       32.52


agriculture_pct

In [ ]:
agri_median = train_pct.groupby('country_code')['agriculture_pct'].agg(lambda x: x.median()).to_dict()
agri_median

{'ARG': 31.444818069405382,
 'AUT': 25.4769494027531,
 'BEL': 26.668181358320435,
 'BGR': 25.155816665267285,
 'CAN': 25.09424120424596,
 'CHE': 26.06140650803472,
 'CZE': 23.15788043170567,
 'DNK': 26.491851914639128,
 'ESP': 29.1957882732232,
 'EST': 22.59304523044876,
 'FIN': 24.857832862389373,
 'FRA': 26.95840670298584,
 'GBR': 25.942266330507906,
 'GRC': 26.55450819315717,
 'HRV': 24.40003847637044,
 'HUN': 24.392846258402848,
 'IND': 25.27138714432398,
 'IRL': 25.772258065710023,
 'ISL': 26.4364270884526,
 'ITA': 26.39715084369472,
 'LTU': 26.039780799234684,
 'LVA': 24.329683235468234,
 'MEX': 22.84812628850013,
 'MKD': 25.00920570732644,
 'NLD': 26.31290080832469,
 'NOR': 27.577072336454936,
 'POL': 24.291803471636847,
 'PRT': 31.53174607099526,
 'ROU': 24.55318570550664,
 'SRB': 24.138820120000982,
 'SVN': 24.682789272820177,
 'SWE': 25.71618644954724,
 'URY': 30.27612064890538,
 'USA': 26.74295287449577}

In [ ]:
train_pct['agriculture_pct'] = train_pct['agriculture_pct'].fillna(train_pct['country_code'].map(agri_median))
test_pct['agriculture_pct'] = test_pct['agriculture_pct'].fillna(test_pct['country_code'].map(agri_median))
data_test_pct['agriculture_pct'] = data_test_pct['agriculture_pct'].fillna(data_test_pct['country_code'].map(agri_median))

In [ ]:
print(pd.concat([
    train_pct[pct_cols].isnull().sum(),
    (train_pct[pct_cols].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                 Missing count  Percentage
humidity_pct                 0        0.00
urban_pct                    0        0.00
forest_pct                   0        0.00
agriculture_pct              0        0.00
water_pct                 3227       28.82
wetland_pct               2713       24.23
grassland_pct             2072       18.50
barren_pct                3642       32.52


water_pct

In [ ]:
water_median = train_pct.groupby('country_code')['water_pct'].agg(lambda x: x.median()).to_dict()
water_median

{'ARG': 6.1110155586508785,
 'AUT': 6.2324433137921975,
 'BEL': 4.524306013194784,
 'BGR': nan,
 'CAN': 10.67698640525782,
 'CHE': 5.000709360783721,
 'CZE': 6.70131100555174,
 'DNK': 5.803930963080014,
 'ESP': 5.315804327102838,
 'EST': 8.27523701568329,
 'FIN': 7.695555606604662,
 'FRA': 4.575911698958836,
 'GBR': 4.390248212805462,
 'GRC': 8.51280381854174,
 'HRV': 6.474741210727461,
 'HUN': 8.00834600237982,
 'IND': 3.85419057975457,
 'IRL': 4.750602927512664,
 'ISL': nan,
 'ITA': 5.65287746258078,
 'LTU': 8.121181200712336,
 'LVA': 7.546903710262048,
 'MEX': 7.025164586780318,
 'MKD': 8.642328062038363,
 'NLD': 4.507104479211143,
 'NOR': 4.026430498405389,
 'POL': 7.184256832603497,
 'PRT': nan,
 'ROU': 8.7941915623195,
 'SRB': 8.215953435247714,
 'SVN': 7.359079590593205,
 'SWE': 5.848139274090998,
 'URY': 8.305920693341424,
 'USA': 5.682646342433434}

In [ ]:
train_pct['water_pct'] = train_pct['water_pct'].fillna(train_pct['country_code'].map(water_median))
test_pct['water_pct'] = test_pct['water_pct'].fillna(test_pct['country_code'].map(water_median))
data_test_pct['water_pct'] = data_test_pct['water_pct'].fillna(data_test_pct['country_code'].map(water_median))

In [ ]:
water_median = train_pct['water_pct'].median()
train_pct['water_pct'] = train_pct['water_pct'].fillna(water_median)
test_pct['water_pct'] = test_pct['water_pct'].fillna(water_median)
data_test_pct['water_pct'] = data_test_pct['water_pct'].fillna(water_median)

In [ ]:
print(pd.concat([
    train_pct[pct_cols].isnull().sum(),
    (train_pct[pct_cols].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                 Missing count  Percentage
humidity_pct                 0        0.00
urban_pct                    0        0.00
forest_pct                   0        0.00
agriculture_pct              0        0.00
water_pct                    0        0.00
wetland_pct               2713       24.23
grassland_pct             2072       18.50
barren_pct                3642       32.52


wetland_pct

In [ ]:
wetland_median = train_pct.groupby('country_code')['wetland_pct'].agg(lambda x: x.median()).to_dict()
wetland_median

{'ARG': 7.263673307120014,
 'AUT': 9.355963797759362,
 'BEL': 10.683311081555088,
 'BGR': nan,
 'CAN': 6.6266294036520055,
 'CHE': 10.865047993452844,
 'CZE': 9.083241371077378,
 'DNK': 9.022489972425438,
 'ESP': 11.548687779916843,
 'EST': 5.42915449785737,
 'FIN': 5.143509931558722,
 'FRA': 10.889004560352362,
 'GBR': 11.03236356180005,
 'GRC': 6.832838385117736,
 'HRV': 8.796600673273131,
 'HUN': 7.924525606929746,
 'IND': 8.442981123598358,
 'IRL': 9.74143105374506,
 'ISL': 5.308113811171887,
 'ITA': 10.178563326315405,
 'LTU': 6.196220035520165,
 'LVA': 6.516206983824295,
 'MEX': 8.392722901531819,
 'MKD': 7.706576257154787,
 'NLD': 10.349754070325943,
 'NOR': 8.083816154330803,
 'POL': 8.011320640320667,
 'PRT': 10.42803175016276,
 'ROU': 6.150704360818924,
 'SRB': 7.505344681203482,
 'SVN': 9.170800894603202,
 'SWE': 7.342751306939446,
 'URY': 4.232182078900221,
 'USA': 8.84116639440198}

In [ ]:
train_pct['wetland_pct'] = train_pct['wetland_pct'].fillna(train_pct['country_code'].map(wetland_median))
test_pct['wetland_pct'] = test_pct['wetland_pct'].fillna(test_pct['country_code'].map(wetland_median))
data_test_pct['wetland_pct'] = data_test_pct['wetland_pct'].fillna(data_test_pct['country_code'].map(wetland_median))

In [ ]:
wetland_median = train_pct['wetland_pct'].median()
train_pct['wetland_pct'] = train_pct['wetland_pct'].fillna(wetland_median)
test_pct['wetland_pct'] = test_pct['wetland_pct'].fillna(wetland_median)
data_test_pct['wetland_pct'] = data_test_pct['wetland_pct'].fillna(wetland_median)

In [ ]:
print(pd.concat([
    train_pct[pct_cols].isnull().sum(),
    (train_pct[pct_cols].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                 Missing count  Percentage
humidity_pct                 0        0.00
urban_pct                    0        0.00
forest_pct                   0        0.00
agriculture_pct              0        0.00
water_pct                    0        0.00
wetland_pct                  0        0.00
grassland_pct             2072       18.50
barren_pct                3642       32.52


grassland_pct

In [ ]:
grassland_median = train_pct.groupby('country_code')['grassland_pct'].agg(lambda x: x.median()).to_dict()
grassland_median

{'ARG': 8.78641040855072,
 'AUT': 14.789369052383371,
 'BEL': 16.833333978155746,
 'BGR': 11.415446328083195,
 'CAN': 16.146107548963556,
 'CHE': 15.568993175175024,
 'CZE': 15.57377091318694,
 'DNK': 18.05796761385958,
 'ESP': 11.741492555815164,
 'EST': 16.692753259216637,
 'FIN': 15.950080336652125,
 'FRA': 14.983941372095655,
 'GBR': 17.234841701340866,
 'GRC': 9.07029985657766,
 'HRV': 13.001916684701657,
 'HUN': 14.081486187982351,
 'IND': 9.448590120682114,
 'IRL': 17.11484923311327,
 'ISL': 12.773134493221532,
 'ITA': 13.588641375316378,
 'LTU': 16.234230891852253,
 'LVA': 16.942095163874853,
 'MEX': 9.520619992013634,
 'MKD': 10.455777304213829,
 'NLD': 17.310270490348223,
 'NOR': 16.08573140322588,
 'POL': 14.979108606273448,
 'PRT': 11.892236714266954,
 'ROU': 13.452220051645575,
 'SRB': 12.167073072780589,
 'SVN': 12.413879079666634,
 'SWE': 16.694571388887812,
 'URY': 8.512906544542474,
 'USA': 9.3507299137141}

In [ ]:
train_pct['grassland_pct'] = train_pct['grassland_pct'].fillna(train_pct['country_code'].map(grassland_median))
test_pct['grassland_pct'] = test_pct['grassland_pct'].fillna(test_pct['country_code'].map(grassland_median))
data_test_pct['grassland_pct'] = data_test_pct['grassland_pct'].fillna(data_test_pct['country_code'].map(grassland_median))

barren_pct

In [ ]:
barren_median = train_pct.groupby('country_code')['barren_pct'].agg(lambda x: x.median()).to_dict()
barren_median

{'ARG': 4.019277118575609,
 'AUT': 6.25759139426467,
 'BEL': 6.818524950086442,
 'BGR': 6.037113722251185,
 'CAN': 5.667857527017895,
 'CHE': 6.441770052492396,
 'CZE': 6.521812729715448,
 'DNK': 5.8655409251785375,
 'ESP': 6.698148932136211,
 'EST': 4.336283540682845,
 'FIN': 3.895440763874177,
 'FRA': 6.429283768061556,
 'GBR': 6.099611182719768,
 'GRC': 6.059564499856189,
 'HRV': 6.649092699846355,
 'HUN': 6.324364127034421,
 'IND': 3.3969591938796424,
 'IRL': 5.285313966022098,
 'ISL': nan,
 'ITA': 6.469467135394397,
 'LTU': 5.334118750389569,
 'LVA': 5.008072446584749,
 'MEX': 4.8580651265121375,
 'MKD': 6.485850883635336,
 'NLD': 6.271442615151233,
 'NOR': 5.2813978354924895,
 'POL': 5.487560813747689,
 'PRT': nan,
 'ROU': 5.886187537183072,
 'SRB': 6.360071089912753,
 'SVN': 6.334317004873062,
 'SWE': 4.8025084510686575,
 'URY': 4.664214240255392,
 'USA': 5.29237804360094}

In [ ]:
train_pct['barren_pct'] = train_pct['barren_pct'].fillna(train_pct['country_code'].map(barren_median))
test_pct['barren_pct'] = test_pct['barren_pct'].fillna(test_pct['country_code'].map(barren_median))
data_test_pct['barren_pct'] = data_test_pct['barren_pct'].fillna(data_test_pct['country_code'].map(barren_median))

In [ ]:
barren_median = train_pct['barren_pct'].median()
train_pct['barren_pct'] = train_pct['barren_pct'].fillna(barren_median)
test_pct['barren_pct'] = test_pct['barren_pct'].fillna(barren_median)
data_test_pct['barren_pct'] = data_test_pct['barren_pct'].fillna(barren_median)

In [ ]:
print(pd.concat([
    train_pct[pct_cols].isnull().sum(),
    (train_pct[pct_cols].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                 Missing count  Percentage
humidity_pct                 0         0.0
urban_pct                    0         0.0
forest_pct                   0         0.0
agriculture_pct              0         0.0
water_pct                    0         0.0
wetland_pct                  0         0.0
grassland_pct                0         0.0
barren_pct                   0         0.0


In [ ]:
print(pd.concat([
    train_pct.isnull().sum(),
    (train_pct.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                          3550       31.70
Parameter Code                 1365       12.19
Dataset_Source                 4967       44.36
temperature_avg                3610       32.24
rainfall_mm                    4182       37.35
humidity_pct                      0        0.00
pressure_hPa                   5462       48.78
drought_index                  4808       42.94
extreme_precip_days            1641       14.65
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                     1225       10.94
soil_perm                      2044     

depth / water_body_type

In [ ]:
train_depth = train_pct.copy()
test_depth = test_pct.copy()
data_test_depth = data_test_pct.copy()

In [ ]:
depth_cols = ['Depth', 'water_body_type']

In [ ]:
print(pd.concat([
    train_depth[depth_cols].isnull().sum(),
    (train_depth[depth_cols].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                 Missing count  Percentage
Depth                     3550        31.7
water_body_type           5341        47.7


In [ ]:
train_depth[depth_cols].head()

,Depth,water_body_type
5474,0.3,NaN
2656,0.5,Reservoir
12646,NaN,NaN
4496,0.3,NaN
13297,NaN,NaN


In [ ]:
train_depth.groupby('water_body_type')['Depth'].median().reset_index()

,water_body_type,Depth
0,Estuary,0.3
1,Groundwater,0.3
2,Lake,0.3
3,Marine,0.3
4,Reservoir,0.3
5,River,0.3
6,Stream,0.3
7,Wetland,0.3


In [ ]:
train_depth['Depth'] = train_depth['Depth'].fillna(0.3)
test_depth['Depth'] = test_depth['Depth'].fillna(0.3)
data_test_depth['Depth'] = data_test_depth['Depth'].fillna(0.3)

In [ ]:
print(pd.concat([
    train_depth.isnull().sum(),
    (train_depth.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                             0        0.00
Parameter Code                 1365       12.19
Dataset_Source                 4967       44.36
temperature_avg                3610       32.24
rainfall_mm                    4182       37.35
humidity_pct                      0        0.00
pressure_hPa                   5462       48.78
drought_index                  4808       42.94
extreme_precip_days            1641       14.65
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                     1225       10.94
soil_perm                      2044     

Parameter Code / param_category

In [ ]:
train_param = train_depth.copy()
test_param = test_depth.copy()
data_test_param = data_test_depth.copy()

For Parameter Code, since the missing values account for only 12%, we can first try imputing them with the mode of param_category, and assign the rest to ‘Unknown’

In [ ]:
param_dict = train_param.groupby('param_category')['Parameter Code'].agg(lambda x: x.mode()).to_dict()
param_dict

{'Common Ion': 'Cl-Dis',
 'Heavy Metal': 'Cd-Tot',
 'Metal': 'Alk-Tot',
 'Nutrient': 'DOC',
 'Other': 'EC'}

In [ ]:
def fill_param(row):
  if pd.isna(row['param_category']):
    return 'Unknown'
  return param_dict.get(row['param_category'], 'Unknown')

In [ ]:
train_param['Parameter Code'] = train_param.apply(
    lambda row: fill_param(row) if pd.isna(row['Parameter Code']) else row['Parameter Code'],
    axis=1
)

test_param['Parameter Code'] = test_param.apply(
    lambda row: fill_param(row) if pd.isna(row['Parameter Code']) else row['Parameter Code'],
    axis=1
)

data_test_param['Parameter Code'] = data_test_param.apply(
    lambda row: fill_param(row) if pd.isna(row['Parameter Code']) else row['Parameter Code'],
    axis=1
)

Since Parameter Code has high cardinality, we will replace it with param_category to simplify the feature

In [ ]:
map_param = train_param.dropna(subset=['param_category', 'Parameter Code']).drop_duplicates('Parameter Code').set_index('Parameter Code')['param_category']
map_param.head()

,param_category
Parameter Code,
HCO3,Common Ion
Ni-Dis,Heavy Metal
Mn-Tot,Metal
Co-Tot,Other
DOC,Nutrient


In [ ]:
train_param['param_category'] = train_param['Parameter Code'].map(map_param)
test_param['param_category'] = test_param['Parameter Code'].map(map_param)
data_test_param['param_category'] = data_test_param['Parameter Code'].map(map_param)

For entries where param_category is unknown, we will assign them to the category “Other”

In [ ]:
train_param['param_category'] = train_param['param_category'].fillna('Other')
test_param['param_category'] = test_param['param_category'].fillna('Other')
data_test_param['param_category'] = data_test_param['param_category'].fillna('Other')

In [ ]:
train_param.drop('Parameter Code', axis=1, inplace=True)
test_param.drop('Parameter Code', axis=1, inplace=True)
data_test_param.drop('Parameter Code', axis=1, inplace=True)

In [ ]:
print(pd.concat([
    train_param.isnull().sum(),
    (train_param.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                             0        0.00
Dataset_Source                 4967       44.36
temperature_avg                3610       32.24
rainfall_mm                    4182       37.35
humidity_pct                      0        0.00
pressure_hPa                   5462       48.78
drought_index                  4808       42.94
extreme_precip_days            1641       14.65
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                     1225       10.94
soil_perm                      2044       18.25
ind_count                      2405     

Dataset_Source

In [ ]:
train_param['Dataset_Source'] = train_param['Dataset_Source'].fillna('Missing_Source')
test_param['Dataset_Source'] = test_param['Dataset_Source'].fillna('Missing_Source')
data_test_param['Dataset_Source'] = data_test_param['Dataset_Source'].fillna('Missing_Source')

In [ ]:
print(pd.concat([
    train_param.isnull().sum(),
    (train_param.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                             0        0.00
Dataset_Source                    0        0.00
temperature_avg                3610       32.24
rainfall_mm                    4182       37.35
humidity_pct                      0        0.00
pressure_hPa                   5462       48.78
drought_index                  4808       42.94
extreme_precip_days            1641       14.65
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                     1225       10.94
soil_perm                      2044       18.25
ind_count                      2405     

temperature_avg / season

In [ ]:
temp_median = train_param.groupby('season')['temperature_avg'].agg(lambda x: x.median()).to_dict()
temp_median

{'Fall': -2.08184845896175,
 'Spring': 14.143601152150126,
 'Summer': 3.8019118508290486,
 'Winter': 13.361731993356091}

In [ ]:
train_param['temperature_avg'] = train_param['temperature_avg'].fillna(train_param['season'].map(temp_median))
test_param['temperature_avg'] = test_param['temperature_avg'].fillna(test_param['season'].map(temp_median))
data_test_param['temperature_avg'] = data_test_param['temperature_avg'].fillna(data_test_param['season'].map(temp_median))

In [ ]:
print(pd.concat([
    train_param.isnull().sum(),
    (train_param.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                             0        0.00
Dataset_Source                    0        0.00
temperature_avg                   0        0.00
rainfall_mm                    4182       37.35
humidity_pct                      0        0.00
pressure_hPa                   5462       48.78
drought_index                  4808       42.94
extreme_precip_days            1641       14.65
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                     1225       10.94
soil_perm                      2044       18.25
ind_count                      2405     

rainfall_mm / season

In [ ]:
rainfall_median = train_param.groupby('season')['rainfall_mm'].agg(lambda x: x.median()).to_dict()
rainfall_median

{'Fall': 53.96897763881893,
 'Spring': 104.06293480509215,
 'Summer': 67.34397881662733,
 'Winter': 95.88537589135768}

In [ ]:
train_param['rainfall_mm'] = train_param['rainfall_mm'].fillna(train_param['season'].map(rainfall_median))
test_param['rainfall_mm'] = test_param['rainfall_mm'].fillna(test_param['season'].map(rainfall_median))
data_test_param['rainfall_mm'] = data_test_param['rainfall_mm'].fillna(data_test_param['season'].map(rainfall_median))

In [ ]:
print(pd.concat([
    train_param.isnull().sum(),
    (train_param.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                             0        0.00
Dataset_Source                    0        0.00
temperature_avg                   0        0.00
rainfall_mm                       0        0.00
humidity_pct                      0        0.00
pressure_hPa                   5462       48.78
drought_index                  4808       42.94
extreme_precip_days            1641       14.65
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                     1225       10.94
soil_perm                      2044       18.25
ind_count                      2405     

pressure_hPa / season

In [ ]:
pressure_median = train_param.groupby('season')['pressure_hPa'].agg(lambda x: x.median()).to_dict()
pressure_median

{'Fall': 1010.6104985723421,
 'Spring': 1006.3011676522516,
 'Summer': 1009.3933556623248,
 'Winter': 1007.1505538512263}

In [ ]:
train_param['pressure_hPa'] = train_param['pressure_hPa'].fillna(train_param['season'].map(pressure_median))
test_param['pressure_hPa'] = test_param['pressure_hPa'].fillna(test_param['season'].map(pressure_median))
data_test_param['pressure_hPa'] = data_test_param['pressure_hPa'].fillna(data_test_param['season'].map(pressure_median))

In [ ]:
print(pd.concat([
    train_param.isnull().sum(),
    (train_param.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                             0        0.00
Dataset_Source                    0        0.00
temperature_avg                   0        0.00
rainfall_mm                       0        0.00
humidity_pct                      0        0.00
pressure_hPa                      0        0.00
drought_index                  4808       42.94
extreme_precip_days            1641       14.65
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                     1225       10.94
soil_perm                      2044       18.25
ind_count                      2405     

drought_index / season

In [ ]:
drought_median = train_param.groupby('season')['drought_index'].agg(lambda x: x.median()).to_dict()
drought_median

{'Fall': -0.0208972429351088,
 'Spring': -0.045018779125664155,
 'Summer': 0.0324046026835502,
 'Winter': 0.0707571742095889}

In [ ]:
train_param['drought_index'] = train_param['drought_index'].fillna(train_param['season'].map(drought_median))
test_param['drought_index'] = test_param['drought_index'].fillna(test_param['season'].map(drought_median))
data_test_param['drought_index'] = data_test_param['drought_index'].fillna(data_test_param['season'].map(drought_median))

In [ ]:
print(pd.concat([
    train_param.isnull().sum(),
    (train_param.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                             0        0.00
Dataset_Source                    0        0.00
temperature_avg                   0        0.00
rainfall_mm                       0        0.00
humidity_pct                      0        0.00
pressure_hPa                      0        0.00
drought_index                     0        0.00
extreme_precip_days            1641       14.65
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                     1225       10.94
soil_perm                      2044       18.25
ind_count                      2405     

extreme_precip_days / season

In [ ]:
extreme_median = train_param.groupby('season')['extreme_precip_days'].agg(lambda x: x.median()).to_dict()
extreme_median

{'Fall': 0.0, 'Spring': 2.0, 'Summer': 1.0, 'Winter': 2.0}

In [ ]:
train_param['extreme_precip_days'] = train_param['extreme_precip_days'].fillna(train_param['season'].map(extreme_median))
test_param['extreme_precip_days'] = test_param['extreme_precip_days'].fillna(test_param['season'].map(extreme_median))
data_test_param['extreme_precip_days'] = data_test_param['extreme_precip_days'].fillna(data_test_param['season'].map(extreme_median))

In [ ]:
print(pd.concat([
    train_param.isnull().sum(),
    (train_param.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                             0        0.00
Dataset_Source                    0        0.00
temperature_avg                   0        0.00
rainfall_mm                       0        0.00
humidity_pct                      0        0.00
pressure_hPa                      0        0.00
drought_index                     0        0.00
extreme_precip_days               0        0.00
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                     1225       10.94
soil_perm                      2044       18.25
ind_count                      2405     

## sampling_method / is_referenced_method / validated_results

In [ ]:
train_rand = train_param.copy()
test_rand = test_param.copy()
data_test_rand = data_test_param.copy()

In [ ]:
rand_col = ['sampling_method', 'is_referenced_method', 'validated_results', 'testing_laboratory']

In [ ]:
train_rand[rand_col].head()

,sampling_method,is_referenced_method,validated_results,testing_laboratory
5474,Composite Sample,False,True,International Water Quality Lab
2656,NaN,False,True,International Water Quality Lab
12646,NaN,False,True,Provincial Analytical Lab
4496,Automated Sample,True,True,Provincial Analytical Lab
13297,Composite Sample,False,False,International Water Quality Lab


In [ ]:
categorical_rand_cols = [col for col in rand_col if train_rand[col].dtype == 'object' or train_rand[col].nunique() < 20]

results = []

for col1, col2 in combinations(categorical_rand_cols, 2):
    df_valid = train_rand[[col1, col2]].dropna()
    if df_valid.empty:
        continue
    v = cramers_v(df_valid[col1], df_valid[col2])
    results.append((col1, col2, round(v, 3)))

pd.DataFrame(results, columns=['Column 1', 'Column 2', "Cramér's V"]).sort_values(by="Cramér's V", ascending=False)

,Column 1,Column 2,Cramér's V
0,sampling_method,is_referenced_method,0.033
1,sampling_method,validated_results,0.016
5,validated_results,testing_laboratory,0.016
3,is_referenced_method,validated_results,0.009
2,sampling_method,testing_laboratory,0.000
4,is_referenced_method,testing_laboratory,0.000


In [ ]:
for col in categorical_rand_cols:
    mode_val = train_rand[col].mode()[0]  # Ambil modus dari train
    train_rand[col] = train_rand[col].fillna(mode_val)
    test_rand[col] = test_rand[col].fillna(mode_val)
    data_test_rand[col] = data_test_rand[col].fillna(mode_val)

### sampling_method

In [ ]:
print(pd.concat([
    train_rand[rand_col].isnull().sum(),
    (train_rand[rand_col].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
sampling_method                   0         0.0
is_referenced_method              0         0.0
validated_results                 0         0.0
testing_laboratory                0         0.0


### is_referenced_method / vlaidated_results

In [ ]:
print(pd.concat([
    train_rand.isnull().sum(),
    (train_rand.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                             0        0.00
Dataset_Source                    0        0.00
temperature_avg                   0        0.00
rainfall_mm                       0        0.00
humidity_pct                      0        0.00
pressure_hPa                      0        0.00
drought_index                     0        0.00
extreme_precip_days               0        0.00
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                     1225       10.94
soil_perm                      2044       18.25
ind_count                      2405     

### sampling_weather / season / month

In [ ]:
train_samp = train_rand.copy()
test_samp = test_rand.copy()
data_test_samp = data_test_rand.copy()

In [ ]:
samp_col = ['sampling_weather', 'season']

In [ ]:
train_samp[samp_col].head()

,sampling_weather,season
5474,NaN,Fall
2656,Rainy,Spring
12646,Cloudy,Spring
4496,Snowy,Summer
13297,Cloudy,Spring


In [ ]:
categorical_samp_cols = [col for col in samp_col if train_samp[col].dtype == 'object' or train_samp[col].nunique() < 20]

results = []

for col1, col2 in combinations(categorical_samp_cols, 2):
    df_valid = train_samp[[col1, col2]].dropna()
    if df_valid.empty:
        continue
    v = cramers_v(df_valid[col1], df_valid[col2])
    results.append((col1, col2, round(v, 3)))

pd.DataFrame(results, columns=['Column 1', 'Column 2', "Cramér's V"]).sort_values(by="Cramér's V", ascending=False)

,Column 1,Column 2,Cramér's V
0,sampling_weather,season,0.255


In [ ]:
map_weather = train_samp.dropna(subset=['season']).groupby('season')['sampling_weather'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan).to_dict()
map_weather

{'Fall': 'Stormy', 'Spring': 'Rainy', 'Summer': 'Clear', 'Winter': 'Snowy'}

In [ ]:
train_samp['sampling_weather'] = train_samp['sampling_weather'].fillna(train_samp['season'].map(map_weather))
test_samp['sampling_weather'] = test_samp['sampling_weather'].fillna(test_samp['season'].map(map_weather))
data_test_samp['sampling_weather'] = data_test_samp['sampling_weather'].fillna(data_test_samp['season'].map(map_weather))

In [ ]:
print(pd.concat([
    train_samp[samp_col].isnull().sum(),
    (train_samp[samp_col].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                  Missing count  Percentage
sampling_weather              0         0.0
season                        0         0.0


In [ ]:
print(pd.concat([
    train_samp.isnull().sum(),
    (train_samp.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                             0        0.00
Dataset_Source                    0        0.00
temperature_avg                   0        0.00
rainfall_mm                       0        0.00
humidity_pct                      0        0.00
pressure_hPa                      0        0.00
drought_index                     0        0.00
extreme_precip_days               0        0.00
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                     1225       10.94
soil_perm                      2044       18.25
ind_count                      2405     

### land_use_impact

In [ ]:
train_samp[['land_use_impact', 'country_code']].head()

,land_use_impact,country_code
5474,Mixed Land Use,IND
2656,Forest Dominant,NLD
12646,Forest Dominant,CAN
4496,Agricultural Dominant,CAN
13297,Agricultural Dominant,MEX


In [ ]:
map_lui = train_samp.groupby('country_code')['land_use_impact'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan).to_dict()
map_lui

{'ARG': 'Forest Dominant',
 'AUT': 'Mixed Land Use',
 'BEL': 'Forest Dominant',
 'BGR': 'Urban Dominant',
 'CAN': 'Forest Dominant',
 'CHE': 'Urban Dominant',
 'CZE': 'Forest Dominant',
 'DNK': 'Forest Dominant',
 'ESP': 'Agricultural Dominant',
 'EST': 'Forest Dominant',
 'FIN': 'Agricultural Dominant',
 'FRA': 'Urban Dominant',
 'GBR': 'Urban Dominant',
 'GRC': 'Mixed Land Use',
 'HRV': 'Urban Dominant',
 'HUN': 'Urban Dominant',
 'IND': 'Forest Dominant',
 'IRL': 'Urban Dominant',
 'ISL': 'Forest Dominant',
 'ITA': 'Mixed Land Use',
 'LTU': 'Forest Dominant',
 'LVA': 'Urban Dominant',
 'MEX': 'Mixed Land Use',
 'MKD': 'Agricultural Dominant',
 'NLD': 'Agricultural Dominant',
 'NOR': 'Urban Dominant',
 'POL': 'Agricultural Dominant',
 'PRT': 'Forest Dominant',
 'ROU': 'Agricultural Dominant',
 'SRB': 'Mixed Land Use',
 'SVN': 'Agricultural Dominant',
 'SWE': 'Forest Dominant',
 'URY': 'Urban Dominant',
 'USA': 'Agricultural Dominant'}

In [ ]:
train_samp['land_use_impact'] = train_samp['land_use_impact'].fillna(train_samp['country_code'].map(map_lui))
test_samp['land_use_impact'] = test_samp['land_use_impact'].fillna(test_samp['country_code'].map(map_lui))
data_test_samp['land_use_impact'] = data_test_samp['land_use_impact'].fillna(data_test_samp['country_code'].map(map_lui))

In [ ]:
print(pd.concat([
    train_samp.isnull().sum(),
    (train_samp.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                             0        0.00
Dataset_Source                    0        0.00
temperature_avg                   0        0.00
rainfall_mm                       0        0.00
humidity_pct                      0        0.00
pressure_hPa                      0        0.00
drought_index                     0        0.00
extreme_precip_days               0        0.00
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                     1225       10.94
soil_perm                      2044       18.25
ind_count                      2405     

## pollution_risk

In [ ]:
train_poll = train_samp.copy()
test_poll = test_samp.copy()
data_test_poll = data_test_samp.copy()

In [ ]:
poll_col = ['pollution_risk', 'param_category']

In [ ]:
train_poll[poll_col].head()

,pollution_risk,param_category
5474,NaN,Common Ion
2656,Medium,Heavy Metal
12646,NaN,Metal
4496,NaN,Other
13297,NaN,Nutrient


In [ ]:
categorical_poll_cols = [col for col in poll_col if train_poll[col].dtype == 'object' or train_poll[col].nunique() < 20]

results = []

for col1, col2 in combinations(categorical_poll_cols, 2):
    df_valid = train_poll[[col1, col2]].dropna()
    if df_valid.empty:
        continue
    v = cramers_v(df_valid[col1], df_valid[col2])
    results.append((col1, col2, round(v, 3)))

pd.DataFrame(results, columns=['Column 1', 'Column 2', "Cramér's V"]).sort_values(by="Cramér's V", ascending=False)

,Column 1,Column 2,Cramér's V
0,pollution_risk,param_category,0.363


In [ ]:
train_poll['pollution_risk'].unique()

array([nan, 'Medium', 'Low', 'Very Low', 'High'], dtype=object)

In [ ]:
map_poll = train_poll.groupby('param_category')['pollution_risk'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan).to_dict()
map_poll

{'Common Ion': 'Low',
 'Heavy Metal': 'Medium',
 'Metal': 'Low',
 'Nutrient': 'Low',
 'Other': 'Low'}

In [ ]:
train_poll['pollution_risk'] = train_poll['pollution_risk'].fillna(train_poll['param_category'].map(map_poll))
test_poll['pollution_risk'] = test_poll['pollution_risk'].fillna(test_poll['param_category'].map(map_poll))
data_test_poll['pollution_risk'] = data_test_poll['pollution_risk'].fillna(data_test_poll['param_category'].map(map_poll))

In [ ]:
print(pd.concat([
    train_poll[poll_col].isnull().sum(),
    (train_poll[poll_col].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                Missing count  Percentage
pollution_risk              0         0.0
param_category              0         0.0


In [ ]:
print(pd.concat([
    train_poll.isnull().sum(),
    (train_poll.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                             0        0.00
Dataset_Source                    0        0.00
temperature_avg                   0        0.00
rainfall_mm                       0        0.00
humidity_pct                      0        0.00
pressure_hPa                      0        0.00
drought_index                     0        0.00
extreme_precip_days               0        0.00
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                     1225       10.94
soil_perm                      2044       18.25
ind_count                      2405     

### water_body_type

In [ ]:
categorical_poll_cols = ['country_code', 'water_body_type']

results = []

for col1, col2 in combinations(categorical_poll_cols, 2):
    df_valid = train_poll[[col1, col2]].dropna()
    if df_valid.empty:
        continue
    v = cramers_v(df_valid[col1], df_valid[col2])
    results.append((col1, col2, round(v, 3)))

pd.DataFrame(results, columns=['Column 1', 'Column 2', "Cramér's V"]).sort_values(by="Cramér's V", ascending=False)

,Column 1,Column 2,Cramér's V
0,country_code,water_body_type,0.036


In [ ]:
wbt_mode = train_poll['water_body_type'].mode()[0]
train_poll['water_body_type'] = train_poll['water_body_type'].fillna(wbt_mode)
test_poll['water_body_type'] = test_poll['water_body_type'].fillna(wbt_mode)
data_test_poll['water_body_type'] = data_test_poll['water_body_type'].fillna(wbt_mode)

In [ ]:
print(pd.concat([
    train_poll.isnull().sum(),
    (train_poll.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                             0        0.00
Dataset_Source                    0        0.00
temperature_avg                   0        0.00
rainfall_mm                       0        0.00
humidity_pct                      0        0.00
pressure_hPa                      0        0.00
drought_index                     0        0.00
extreme_precip_days               0        0.00
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                     1225       10.94
soil_perm                      2044       18.25
ind_count                      2405     

analytical_program

In [ ]:
categorical_poll_cols = ['regulatory_framework', 'analytical_program']

results = []

for col1, col2 in combinations(categorical_poll_cols, 2):
    df_valid = train_poll[[col1, col2]].dropna()
    if df_valid.empty:
        continue
    v = cramers_v(df_valid[col1], df_valid[col2])
    results.append((col1, col2, round(v, 3)))

pd.DataFrame(results, columns=['Column 1', 'Column 2', "Cramér's V"]).sort_values(by="Cramér's V", ascending=False)

,Column 1,Column 2,Cramér's V
0,regulatory_framework,analytical_program,0.547


In [ ]:
analy_mode = train_poll.groupby('regulatory_framework')['analytical_program'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan).to_dict()
analy_mode

{'Baltic Sea Protection Convention': 'National River Assessment',
 'Canadian Environmental Protection Act': 'Provincial Monitoring',
 'EU Water Framework Directive': 'Mediterranean Basin Assessment',
 'Fisheries Act': 'Great Lakes Assessment',
 'Greek Environmental Law': 'EU WFD Implementation',
 'International Water Quality Standards': 'International Water Assessment',
 'Mediterranean Action Plan': 'Hellenic Water Monitoring',
 'Polish Water Law': 'National River Assessment',
 'Provincial Water Regulations': 'Great Lakes Assessment'}

In [ ]:
train_poll['analytical_program'] = train_poll['analytical_program'].fillna(train_poll['regulatory_framework'].map(analy_mode))
test_poll['analytical_program'] = test_poll['analytical_program'].fillna(test_poll['regulatory_framework'].map(analy_mode))
data_test_poll['analytical_program'] = data_test_poll['analytical_program'].fillna(data_test_poll['regulatory_framework'].map(analy_mode))

In [ ]:
print(pd.concat([
    train_poll.isnull().sum(),
    (train_poll.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                             0        0.00
Dataset_Source                    0        0.00
temperature_avg                   0        0.00
rainfall_mm                       0        0.00
humidity_pct                      0        0.00
pressure_hPa                      0        0.00
drought_index                     0        0.00
extreme_precip_days               0        0.00
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                     1225       10.94
soil_perm                      2044       18.25
ind_count                      2405     

## Some NUMERICAL VALUE

In [ ]:
train_num = train_poll.copy()
test_num = test_poll.copy()
data_test_num = data_test_poll.copy()

In [ ]:
nume_col = ['frag_index', 'soil_perm', 'ind_count', 'hg_impact', 'pb_impact', 'ind_risk_score', "value_ug_per_l"]

In [ ]:
cat

['Station Code',
 'Parameter Code',
 'Dataset_Source',
 'param_category',
 'country_code',
 'time_season',
 'sampling_method',
 'is_referenced_method',
 'water_body_type',
 'analytical_program',
 'pollution_risk',
 'testing_laboratory',
 'validated_results',
 'sampling_weather',
 'regulatory_framework',
 'land_use_impact',
 'Data Quality']

In [ ]:
def correlation_ratio(categories, measurements):
    fcat, _ = pd.factorize(categories)
    cat_num = np.max(fcat) + 1
    y_avg_array = np.zeros(cat_num)
    n_array = np.zeros(cat_num)
    for i in range(0, cat_num):
        cat_measures = measurements[fcat == i]
        n_array[i] = len(cat_measures)
        y_avg_array[i] = np.nanmean(cat_measures)
    y_total_avg = np.nansum(y_avg_array * n_array) / np.nansum(n_array)
    numerator = np.nansum(n_array * (y_avg_array - y_total_avg) ** 2)
    denominator = np.nansum((measurements - y_total_avg) ** 2)
    if denominator == 0:
        return 0.0
    else:
        return numerator / denominator

In [ ]:
# Identify categorical columns
cat_cols = [col for col in train_num.columns if train_num[col].dtype == 'object']

results = []

# Loop: numerical × categorical
for num_col in nume_col:
    for cat_col in cat_cols:
        if cat_col in train_num.columns:
            try:
                score = correlation_ratio(train_num[cat_col], train_num[num_col])
                results.append((num_col, cat_col, round(score, 3)))
            except:
                continue

# Convert to DataFrame
correlation_ratios_df = pd.DataFrame(results, columns=['Numerical', 'Categorical', 'CorrelationRatio (η²)'])
correlation_ratios_df = correlation_ratios_df.sort_values(by='CorrelationRatio (η²)', ascending=False)

correlation_ratios_df

,Numerical,Categorical,CorrelationRatio (η²)
17,soil_perm,country_code,0.571
2,frag_index,country_code,0.567
90,value_ug_per_l,Dataset_Source,0.286
5,frag_index,analytical_program,0.220
9,frag_index,regulatory_framework,0.215
...,...,...,...
61,pb_impact,param_category,0.000
51,hg_impact,pollution_risk,0.000
46,hg_impact,param_category,0.000
57,hg_impact,season,0.000


In [ ]:
top_corr_per_numeric = (
    correlation_ratios_df.sort_values('CorrelationRatio (η²)', ascending=False)
    .groupby('Numerical', as_index=False)
    .first()
)
top_corr_per_numeric

,Numerical,Categorical,CorrelationRatio (η²)
0,frag_index,country_code,0.567
1,hg_impact,Dataset_Source,0.016
2,ind_count,country_code,0.153
3,ind_risk_score,country_code,0.082
4,pb_impact,Dataset_Source,0.064
5,soil_perm,country_code,0.571
6,value_ug_per_l,Dataset_Source,0.286


In [ ]:
train_num[nume_col].head()

,frag_index,soil_perm,ind_count,hg_impact,pb_impact,ind_risk_score,value_ug_per_l
5474,0.769895,NaN,0.0,NaN,0.0,0.0,122000.00
2656,0.711029,0.601835,0.0,0.0,NaN,0.0,2.00
12646,0.808661,0.537999,0.0,0.0,NaN,0.0,7.73
4496,0.663679,0.603280,0.0,NaN,0.0,0.0,0.16
13297,0.788825,0.569108,0.0,NaN,NaN,NaN,NaN


### frag_index

In [ ]:
frag_median = train_num.groupby(['country_code'])['frag_index'].agg(lambda x: x.median()).to_dict()

In [ ]:
train_num['frag_index'] = train_num['frag_index'].fillna(train_num['country_code'].map(frag_median))
test_num['frag_index'] = test_num['frag_index'].fillna(test_num['country_code'].map(frag_median))
data_test_num['frag_index'] = data_test_num['frag_index'].fillna(data_test_num['country_code'].map(frag_median))

In [ ]:
print(pd.concat([
    train_num[nume_col].isnull().sum(),
    (train_num[nume_col].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                Missing count  Percentage
frag_index                  0        0.00
soil_perm                2044       18.25
ind_count                2405       21.48
hg_impact                2851       25.46
pb_impact                4547       40.61
ind_risk_score           1565       13.98
value_ug_per_l           4849       43.30


### soil_perm

In [ ]:
soil_median = train_num.groupby(['country_code'])['soil_perm'].agg(lambda x: x.median()).to_dict()

In [ ]:
train_num['soil_perm'] = train_num['soil_perm'].fillna(train_num['country_code'].map(soil_median))
test_num['soil_perm'] = test_num['soil_perm'].fillna(test_num['country_code'].map(soil_median))
data_test_num['soil_perm'] = data_test_num['soil_perm'].fillna(data_test_num['country_code'].map(soil_median))

In [ ]:
print(pd.concat([
    train_num[nume_col].isnull().sum(),
    (train_num[nume_col].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                Missing count  Percentage
frag_index                  0        0.00
soil_perm                   2        0.02
ind_count                2405       21.48
hg_impact                2851       25.46
pb_impact                4547       40.61
ind_risk_score           1565       13.98
value_ug_per_l           4849       43.30


In [ ]:
soil_median = train_num['soil_perm'].median()
train_num['soil_perm'] = train_num['soil_perm'].fillna(soil_median)
test_num['soil_perm'] = test_num['soil_perm'].fillna(soil_median)
data_test_num['soil_perm'] = data_test_num['soil_perm'].fillna(soil_median)

### ind_count

In [ ]:
ind_median = train_num.groupby(['country_code'])['ind_count'].agg(lambda x: x.median()).to_dict()

In [ ]:
train_num['ind_count'] = train_num['ind_count'].fillna(train_num['country_code'].map(ind_median))
test_num['ind_count'] = test_num['ind_count'].fillna(test_num['country_code'].map(ind_median))
data_test_num['ind_count'] = data_test_num['ind_count'].fillna(data_test_num['country_code'].map(ind_median))

In [ ]:
print(pd.concat([
    train_num[nume_col].isnull().sum(),
    (train_num[nume_col].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                Missing count  Percentage
frag_index                  0        0.00
soil_perm                   0        0.00
ind_count                   0        0.00
hg_impact                2851       25.46
pb_impact                4547       40.61
ind_risk_score           1565       13.98
value_ug_per_l           4849       43.30


### hg_impact

In [ ]:
hg_median = train_num.groupby(['Dataset_Source'])['hg_impact'].agg(lambda x: x.median()).to_dict()

In [ ]:
train_num['hg_impact'] = train_num['hg_impact'].fillna(train_num['country_code'].map(hg_median))
test_num['hg_impact'] = test_num['hg_impact'].fillna(test_num['country_code'].map(hg_median))
data_test_num['hg_impact'] = data_test_num['hg_impact'].fillna(data_test_num['country_code'].map(hg_median))

In [ ]:
print(pd.concat([
    train_num[nume_col].isnull().sum(),
    (train_num[nume_col].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                Missing count  Percentage
frag_index                  0        0.00
soil_perm                   0        0.00
ind_count                   0        0.00
hg_impact                2851       25.46
pb_impact                4547       40.61
ind_risk_score           1565       13.98
value_ug_per_l           4849       43.30


In [ ]:
hg_median = train_num['hg_impact'].median()
train_num['hg_impact'] = train_num['hg_impact'].fillna(hg_median)
test_num['hg_impact'] = test_num['hg_impact'].fillna(hg_median)
data_test_num['hg_impact'] = data_test_num['hg_impact'].fillna(hg_median)

### pb_impact

In [ ]:
pb_median = train_num.groupby(['Dataset_Source'])['pb_impact'].agg(lambda x: x.median()).to_dict()

In [ ]:
train_num['pb_impact'] = train_num['pb_impact'].fillna(train_num['country_code'].map(pb_median))
test_num['pb_impact'] = test_num['pb_impact'].fillna(test_num['country_code'].map(pb_median))
data_test_num['pb_impact'] = data_test_num['pb_impact'].fillna(data_test_num['country_code'].map(pb_median))

In [ ]:
print(pd.concat([
    train_num[nume_col].isnull().sum(),
    (train_num[nume_col].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                Missing count  Percentage
frag_index                  0        0.00
soil_perm                   0        0.00
ind_count                   0        0.00
hg_impact                   0        0.00
pb_impact                4547       40.61
ind_risk_score           1565       13.98
value_ug_per_l           4849       43.30


In [ ]:
pb_median = train_num['pb_impact'].median()
train_num['pb_impact'] = train_num['pb_impact'].fillna(pb_median)
test_num['pb_impact'] = test_num['pb_impact'].fillna(pb_median)
data_test_num['pb_impact'] = data_test_num['pb_impact'].fillna(pb_median)

### ind_risk_score

In [ ]:
ind_risk_median = train_num.groupby(['country_code'])['ind_risk_score'].agg(lambda x: x.median()).to_dict()

In [ ]:
train_num['ind_risk_score'] = train_num['ind_risk_score'].fillna(train_num['country_code'].map(ind_risk_median))
test_num['ind_risk_score'] = test_num['ind_risk_score'].fillna(test_num['country_code'].map(ind_risk_median))
data_test_num['ind_risk_score'] = data_test_num['ind_risk_score'].fillna(data_test_num['country_code'].map(ind_risk_median))

In [ ]:
print(pd.concat([
    train_num[nume_col].isnull().sum(),
    (train_num[nume_col].isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                Missing count  Percentage
frag_index                  0        0.00
soil_perm                   0        0.00
ind_count                   0        0.00
hg_impact                   0        0.00
pb_impact                   0        0.00
ind_risk_score              1        0.01
value_ug_per_l           4849       43.30


In [ ]:
ind_risk_median = train_num['ind_risk_score'].median()
train_num['ind_risk_score'] = train_num['ind_risk_score'].fillna(ind_risk_median)
test_num['ind_risk_score'] = test_num['ind_risk_score'].fillna(ind_risk_median)
data_test_num['ind_risk_score'] = data_test_num['ind_risk_score'].fillna(ind_risk_median)

In [ ]:
# Buat mapping: country_code → median value_ug_per_l
value_median = train_num.groupby('country_code')['value_ug_per_l'].median().to_dict()

# Imputasi nilai yang hilang berdasarkan country_code
train_num['value_ug_per_l'] = train_num['value_ug_per_l'].fillna(train_num['country_code'].map(value_median))
test_num['value_ug_per_l'] = test_num['value_ug_per_l'].fillna(test_num['country_code'].map(value_median))
data_test_num['value_ug_per_l'] = data_test_num['value_ug_per_l'].fillna(data_test_num['country_code'].map(value_median))

In [ ]:
print(pd.concat([
    train_num.isnull().sum(),
    (train_num.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

print(pd.concat([
    test_num.isnull().sum(),
    (test_num.isnull().mean()*100).round(2)
], axis=1, keys=['Missing count', 'Percentage']))

                      Missing count  Percentage
Latitude                          0        0.00
Longitude                         0        0.00
Depth                             0        0.00
Dataset_Source                    0        0.00
temperature_avg                   0        0.00
rainfall_mm                       0        0.00
humidity_pct                      0        0.00
pressure_hPa                      0        0.00
drought_index                     0        0.00
extreme_precip_days               0        0.00
urban_pct                         0        0.00
forest_pct                        0        0.00
agriculture_pct                   0        0.00
water_pct                         0        0.00
wetland_pct                       0        0.00
grassland_pct                     0        0.00
barren_pct                        0        0.00
frag_index                        0        0.00
soil_perm                         0        0.00
ind_count                         0     

# Feature Engineering

In [ ]:
train_eng = train_num.copy()
test_eng = test_num.copy()
data_test_eng = data_test_num.copy()

In [ ]:
train_eng.head()

,Latitude,Longitude,Depth,Dataset_Source,temperature_avg,rainfall_mm,humidity_pct,pressure_hPa,drought_index,extreme_precip_days,...,testing_laboratory,validated_results,sampling_weather,regulatory_framework,land_use_impact,Data Quality,value_ug_per_l,season,year,month
5474,22.735833,79.379167,0.3,Missing_Source,10.536335,63.990494,78.738071,1009.472994,-0.020897,0.0,...,International Water Quality Lab,True,Stormy,International Water Quality Standards,Mixed Land Use,Unknown,122000.000,Fall,1993,11
2656,52.400650,5.699110,0.5,nickel,6.306847,116.040301,94.566138,1007.878868,-0.045019,2.0,...,International Water Quality Lab,True,Rainy,International Water Quality Standards,Forest Dominant,Fair,2.000,Spring,2015,03
12646,50.300833,-116.579400,0.3,manganese,8.904922,105.575786,79.558403,1011.804084,-0.045019,1.0,...,Provincial Analytical Lab,True,Cloudy,Canadian Environmental Protection Act,Forest Dominant,Fair,7.730,Spring,2019,04
4496,44.763600,-116.579400,0.3,cobalt,-6.234326,45.734659,77.847505,1009.393356,0.493044,0.0,...,Provincial Analytical Lab,True,Snowy,Provincial Water Regulations,Agricultural Dominant,Fair,0.160,Summer,2017,08
13297,16.417300,-91.263770,0.3,carbon,24.995495,73.492209,65.971375,1006.301168,-1.190537,2.0,...,International Water Quality Lab,False,Cloudy,International Water Quality Standards,Agricultural Dominant,Fair,4.989,Spring,2015,04


In [ ]:
cat, num = cat_num(train_eng.drop('Data Quality', axis=1))

In [ ]:
train_eng['Data Quality'].value_counts()

,count
Data Quality,
Fair,8106
Unknown,2872
Good,201
Pending review,19


## Categorical Target Encoding

For categorical features, we will apply target encoding by using the target variable. In this case, we take the least frequent class, “Pending Review”, and encode categories based on their probability of belonging to that class

In [ ]:
cat.remove('year')
cat.remove('month')
cat

['Dataset_Source',
 'param_category',
 'country_code',
 'sampling_method',
 'water_body_type',
 'analytical_program',
 'pollution_risk',
 'testing_laboratory',
 'sampling_weather',
 'regulatory_framework',
 'land_use_impact',
 'season']

In [ ]:
for i in cat:
  print(i, train_eng[i].nunique())
  print(train_eng[i].value_counts())

Dataset_Source 39
Dataset_Source
Missing_Source            4967
dissolved_gas              593
carbon                     448
electrical_conductance     387
indicator_organism         369
chloride                   302
nickel                     269
calcium                    257
lead                       256
magnesium                  253
hardness                   250
chromium                   237
cadmium                    224
halocarbon                 219
alkalinity                 214
arsenic                    213
iron                       211
copper                     185
bicarbonate                182
mercury                    160
manganese                  120
aluminium                  117
fluoride                   102
boron                       95
cobalt                      67
molybdenum                  61
cyanide                     56
barium                      52
flux                        49
berillium                   46
antimony                    43
lithiu

Categorical columns are transformed into numerical representations, where each category is encoded based on its probability of belonging to Data Quality = ‘Pending Review’

Dataset_Source

Mengubah kolom Dataset_Source (teks) menjadi angka yang mewakili seberapa banyak baris Data Quality = 'Pending review'

In [ ]:
dss_proba = train_eng.groupby('Dataset_Source')['Data Quality'].apply(lambda x: (x == 'Pending review').sum() / len(x))
dss_proba = dss_proba.to_dict()

In [ ]:
train_eng['Dataset_Source'] = train_eng['Dataset_Source'].map(dss_proba)
test_eng['Dataset_Source'] = test_eng['Dataset_Source'].map(dss_proba)
data_test_eng['Dataset_Source'] = data_test_eng['Dataset_Source'].map(dss_proba)

param_category

In [ ]:
pc_proba = train_eng.groupby('param_category')['Data Quality'].apply(lambda x: (x == 'Pending review').sum() / len(x))
pc_proba = pc_proba.to_dict()

In [ ]:
train_eng['param_category'] = train_eng['param_category'].map(pc_proba)
test_eng['param_category'] = test_eng['param_category'].map(pc_proba)
data_test_eng['param_category'] = data_test_eng['param_category'].map(pc_proba)

country_code

In [ ]:
country_proba = train_eng.groupby('country_code')['Data Quality'].apply(lambda x: (x == 'Pending review').sum() / len(x))
country_proba = country_proba.to_dict()

In [ ]:
train_eng['country_code'] = train_eng['country_code'].map(country_proba)
test_eng['country_code'] = test_eng['country_code'].map(country_proba)
data_test_eng['country_code'] = data_test_eng['country_code'].map(country_proba)

sampling_method

In [ ]:
sampling_proba = train_eng.groupby('sampling_method')['Data Quality'].apply(lambda x: (x == 'Pending review').sum() / len(x))
sampling_proba = sampling_proba.to_dict()

In [ ]:
train_eng['sampling_method'] = train_eng['sampling_method'].map(sampling_proba)
test_eng['sampling_method'] = test_eng['sampling_method'].map(sampling_proba)
data_test_eng['sampling_method'] = data_test_eng['sampling_method'].map(sampling_proba)

is_referenced_method

In [ ]:
train_eng['is_referenced_method'] = train_eng['is_referenced_method'].astype(int)
test_eng['is_referenced_method'] = test_eng['is_referenced_method'].astype(int)
data_test_eng['is_referenced_method'] = data_test_eng['is_referenced_method'].astype(int)

water_body_type

In [ ]:
wbt_proba = train_eng.groupby('water_body_type')['Data Quality'].apply(lambda x: (x == 'Pending review').sum() / len(x))
wbt_proba = wbt_proba.to_dict()

In [ ]:
train_eng['water_body_type'] = train_eng['water_body_type'].map(wbt_proba)
test_eng['water_body_type'] = test_eng['water_body_type'].map(wbt_proba)
data_test_eng['water_body_type'] = data_test_eng['water_body_type'].map(wbt_proba)

analytical_program

In [ ]:
ana_prob = train_eng.groupby('analytical_program')['Data Quality'].apply(lambda x: (x == 'Pending review').sum() / len(x))
ana_prob = ana_prob.to_dict()

In [ ]:
train_eng['analytical_program'] = train_eng['analytical_program'].map(ana_prob)
test_eng['analytical_program'] = test_eng['analytical_program'].map(ana_prob)
data_test_eng['analytical_program'] = data_test_eng['analytical_program'].map(ana_prob)

pollution_risk

In [ ]:
train_eng['pollution_risk'].unique()

array(['Low', 'Medium', 'Very Low', 'High'], dtype=object)

In [ ]:
# Buat mapping ordinal berdasarkan tingkat risiko
pollution_risk_map = {
    'Very Low': 0,
    'Low': 1,
    'Medium': 2,
    'High': 3
}

# Terapkan ke semua dataset
train_eng['pollution_risk'] = train_eng['pollution_risk'].map(pollution_risk_map)
test_eng['pollution_risk'] = test_eng['pollution_risk'].map(pollution_risk_map)
data_test_eng['pollution_risk'] = data_test_eng['pollution_risk'].map(pollution_risk_map)

testing_laboratory

In [ ]:
testing_prob = train_eng.groupby('testing_laboratory')['Data Quality'].apply(lambda x: (x == 'Pending review').sum() / len(x))
testing_prob = testing_prob.to_dict()

In [ ]:
train_eng['testing_laboratory'] = train_eng['testing_laboratory'].map(testing_prob)
test_eng['testing_laboratory'] = test_eng['testing_laboratory'].map(testing_prob)
data_test_eng['testing_laboratory'] = data_test_eng['testing_laboratory'].map(testing_prob)

validated_results

In [ ]:
train_eng['validated_results'] = train_eng['validated_results'].astype(int)
test_eng['validated_results'] = test_eng['validated_results'].astype(int)
data_test_eng['validated_results'] = data_test_eng['validated_results'].astype(int)

sampling_weather

In [ ]:
samp_prob = train_eng.groupby('sampling_weather')['Data Quality'].apply(lambda x: (x == 'Pending review').sum() / len(x))
samp_prob = samp_prob.to_dict()

In [ ]:
train_eng['sampling_weather'] = train_eng['sampling_weather'].map(samp_prob)
test_eng['sampling_weather'] = test_eng['sampling_weather'].map(samp_prob)
data_test_eng['sampling_weather'] = data_test_eng['sampling_weather'].map(samp_prob)

regulatory_framework

In [ ]:
regula_proba = train_eng.groupby('regulatory_framework')['Data Quality'].apply(lambda x: (x == 'Pending review').sum() / len(x))
regula_proba = regula_proba.to_dict()

In [ ]:
train_eng['regulatory_framework'] = train_eng['regulatory_framework'].map(regula_proba)
test_eng['regulatory_framework'] = test_eng['regulatory_framework'].map(regula_proba)
data_test_eng['regulatory_framework'] = data_test_eng['regulatory_framework'].map(regula_proba)

land_use_impact

In [ ]:
land_use_map = {
    'Forest Dominant': 0,
    'Agricultural Dominant': 1,
    'Mixed Land Use': 2,
    'Urban Dominant': 3
}

train_eng['land_use_impact'] = train_eng['land_use_impact'].map(land_use_map)
test_eng['land_use_impact'] = test_eng['land_use_impact'].map(land_use_map)
data_test_eng['land_use_impact'] = data_test_eng['land_use_impact'].map(land_use_map)

season

In [ ]:
season_map = {
    'Winter': 0,
    'Spring': 1,
    'Summer': 2,
    'Fall': 3
}

train_eng['season'] = train_eng['season'].map(season_map)
test_eng['season'] = test_eng['season'].map(season_map)
data_test_eng['season'] = data_test_eng['season'].map(season_map)

### Month Year

year / month

In [ ]:
train_eng['year'] = train_eng['year'].astype(int)
test_eng['year'] = test_eng['year'].astype(int)
data_test_eng['year'] = data_test_eng['year'].astype(int)

train_eng['month'] = train_eng['month'].astype(int)
test_eng['month'] = test_eng['month'].astype(int)
data_test_eng['month'] = data_test_eng['month'].astype(int)

month sin cos

In [ ]:
train_eng['month_sin'] = np.sin(2 * np.pi * train_eng['month'] / 12)
test_eng['month_sin'] = np.sin(2 * np.pi * test_eng['month'] / 12)
data_test_eng['month_sin'] = np.sin(2 * np.pi * data_test_eng['month'] / 12)

train_eng['month_cos'] = np.cos(2 * np.pi * train_eng['month'] / 12)
test_eng['month_cos'] = np.cos(2 * np.pi * test_eng['month'] / 12)
data_test_eng['month_cos'] = np.cos(2 * np.pi * data_test_eng['month'] / 12)

In [ ]:
target_le = LabelEncoder()
train_eng['Data Quality'] = target_le.fit_transform(train_eng['Data Quality'])
test_eng['Data Quality'] = target_le.transform(test_eng['Data Quality'])

In [ ]:
train_eng.columns

Index(['Latitude', 'Longitude', 'Depth', 'Dataset_Source', 'temperature_avg',
       'rainfall_mm', 'humidity_pct', 'pressure_hPa', 'drought_index',
       'extreme_precip_days', 'urban_pct', 'forest_pct', 'agriculture_pct',
       'water_pct', 'wetland_pct', 'grassland_pct', 'barren_pct', 'frag_index',
       'soil_perm', 'ind_count', 'hg_impact', 'pb_impact', 'ind_risk_score',
       'param_category', 'country_code', 'sampling_method',
       'is_referenced_method', 'water_body_type', 'analytical_program',
       'pollution_risk', 'testing_laboratory', 'validated_results',
       'sampling_weather', 'regulatory_framework', 'land_use_impact',
       'Data Quality', 'value_ug_per_l', 'season', 'year', 'month',
       'month_sin', 'month_cos'],
      dtype='object')

In [ ]:
corr = train_eng.drop('Data Quality', axis=1).corr().abs()
high_corr_pairs = [(i, j) for i in corr.columns for j in corr.columns
                   if i != j and corr.loc[i, j] > 0.95]
high_corr_pairs

[]

## Transform

In [ ]:
cat, num = cat_num(train_eng.drop('Data Quality', axis=1))

In [ ]:
pt = PowerTransformer()

train_eng[num] = pt.fit_transform(train_eng[num])
test_eng[num] = pt.transform(test_eng[num])
data_test_eng[num] = pt.transform(data_test_eng[num])

In [ ]:
# # Inisialisasi
# rs = RobustScaler()

# # Fit hanya ke train
# train_eng[nume_col] = rs.fit_transform(train_eng[nume_col])
# test_eng[nume_col] = rs.transform(test_eng[nume_col])
# data_test_eng[nume_col] = rs.transform(data_test_eng[nume_col])

In [ ]:
train_eng.head()

,Latitude,Longitude,Depth,Dataset_Source,temperature_avg,rainfall_mm,humidity_pct,pressure_hPa,drought_index,extreme_precip_days,...,sampling_weather,regulatory_framework,land_use_impact,Data Quality,value_ug_per_l,season,year,month,month_sin,month_cos
5474,-0.832960,1.541399,0.112724,0.736869,0.384636,-0.572835,-0.221523,0.350928,-0.035588,-1.220428,...,-1.546323,-0.117634,0.542762,3,1.512529,1.322915,-1.950700,1.400026,-0.709418,1.329852
2656,1.121492,0.457310,1.412726,-1.048892,-0.059773,1.339932,1.639355,-0.207685,-0.063072,0.803164,...,-0.711355,-0.117634,-1.371384,0,-0.782789,-0.505065,0.349012,-1.004409,1.464790,0.219115
12646,0.975665,-1.077105,0.112724,-1.048892,0.213238,1.000321,-0.127194,1.170674,-0.063072,0.095337,...,1.428494,-2.702990,-1.371384,0,-0.393512,-0.505065,0.905497,-0.664770,1.270786,-0.504499
4496,0.595805,-1.077105,0.112724,-1.048892,-1.380101,-1.442220,-0.323653,0.322983,0.550280,-1.220428,...,-0.347946,2.146938,-0.319230,0,-1.175228,0.425615,0.620946,0.564343,-1.240725,-0.504499
13297,-1.210305,-0.767092,0.112724,-1.048892,1.903025,-0.173379,-1.656973,-0.758967,-1.367035,0.803164,...,1.428494,-0.117634,-0.319230,0,-0.524996,-0.505065,0.349012,-0.664770,1.270786,-0.504499


In [ ]:
X = train_eng.drop('Data Quality', axis=1)
y = train_eng['Data Quality']

In [ ]:
xgb = XGBClassifier(random_state=27)
lgbm = LGBMClassifier(random_state=27)

In [ ]:
xgb.fit(X, y)
lgbm.fit(X, y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002875 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5325
[LightGBM] [Info] Number of data points in the train set: 11198, number of used features: 41
[LightGBM] [Info] Start training from score -0.323131
[LightGBM] [Info] Start training from score -4.020186
[LightGBM] [Info] Start training from score -6.379051
[LightGBM] [Info] Start training from score -1.360727
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

LGBMClassifier(random_state=27)

In [ ]:
# from sklearn.feature_selection import RFECV
# from sklearn.model_selection import StratifiedKFold

# cv = StratifiedKFold(n_splits=5)

# selector = RFECV(estimator=xgb, step=1, cv=cv, scoring='f1_weighted', n_jobs=-1)
# selector.fit(X, y)

# selected_features = X.columns[selector.support_].tolist()

# print("Jumlah fitur optimal:", selector.n_features_)
# print("Fitur terpilih:")
# print(selected_features)

In [ ]:
# # Simpan daftar fitur hasil seleksi
# selected_features = ['Latitude', 'Longitude', 'Depth', 'urban_pct', 'forest_pct', 'agriculture_pct', 'water_pct', 'wetland_pct', 'grassland_pct', 'barren_pct', 'soil_perm', 'pb_impact', 'ind_risk_score', 'country_code', 'value_ug_per_l', 'year']

# # Filter hanya fitur yang dipilih
# train_eng = train_eng[selected_features + ['Data Quality']]
# test_eng = test_eng[selected_features + ['Data Quality']]
# data_test_eng = data_test_eng[['id'] + selected_features]

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
vc = VotingClassifier(estimators=[('xgb', xgb), ('lgbm', lgbm)], voting='soft')
vc.fit(train_eng.drop('Data Quality', axis=1), train_eng['Data Quality'])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002457 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5325
[LightGBM] [Info] Number of data points in the train set: 11198, number of used features: 41
[LightGBM] [Info] Start training from score -0.323131
[LightGBM] [Info] Start training from score -4.020186
[LightGBM] [Info] Start training from score -6.379051
[LightGBM] [Info] Start training from score -1.360727
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            objective='multi:softprob', ...)),
                             ('lgbm', LGBMClassifier(random_state=27))],
                 voting='soft')

In [ ]:
lgbm.fit(train_eng.drop('Data Quality', axis=1), train_eng['Data Quality'])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002449 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5325
[LightGBM] [Info] Number of data points in the train set: 11198, number of used features: 41
[LightGBM] [Info] Start training from score -0.323131
[LightGBM] [Info] Start training from score -4.020186
[LightGBM] [Info] Start training from score -6.379051
[LightGBM] [Info] Start training from score -1.360727
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

LGBMClassifier(random_state=27)

In [ ]:
test_eng.isna().sum()

,0
Latitude,70
Longitude,174
Depth,0
Dataset_Source,1
temperature_avg,0
rainfall_mm,0
humidity_pct,23
pressure_hPa,0
drought_index,0
extreme_precip_days,0


In [ ]:
vc_preds = lgbm.predict(test_eng.drop('Data Quality', axis=1))
f1_score(test_eng['Data Quality'], vc_preds, average='macro')

0.6751211445680076

In [ ]:
test_preds = lgbm.predict(data_test_eng.drop('id', axis=1))

In [ ]:
sample_sub = pd.read_csv('sample_submission.csv')

In [ ]:
sample_sub['Data Quality'] = test_preds.flatten()

In [ ]:
sample_sub['Data Quality'] = target_le.inverse_transform(sample_sub['Data Quality'])

In [ ]:
sample_sub['Data Quality'].value_counts()

,count
Data Quality,
Fair,4534
Unknown,1394
Good,69
Pending review,3


In [ ]:
sample_sub.to_csv('12052025_1.csv', index=False)